In [ ]:
import pandas as pd

In [37]:
df = pd.read_parquet(caminho_parquet)
df.head()  # Exibir as primeiras linhas

,loja_id,produto_id,categoria_id,is_medicamento,curva,data,estoque,venda,custo,preco
0,2,5124,2,False,E,2024-11-12,0,0.0,0.111198,0.171087
1,2,208,2,False,D,2023-08-13,3,0.0,0.069276,0.134420
2,7,208,2,False,D,2022-06-14,2,0.0,0.069276,0.124337
3,10,5124,2,False,E,2022-09-01,1,0.0,0.111198,0.171087
4,4,5124,2,False,E,2024-10-11,0,0.0,0.111198,0.171087


In [23]:
df.shape

(41389576, 10)

In [ ]:
print(df.isnull().sum())  # Contagem de valores nulos

loja_id           0
produto_id        0
categoria_id      0
is_medicamento    0
curva             0
data              0
estoque           0
venda             0
custo             0
preco             0
dtype: int64


In [27]:
df.groupby(['loja_id','produto_id']).agg({'data':'count'}).reset_index().sort_values('data')

,loja_id,produto_id,data
15056,4,2528,1
11093,3,2809,1
36805,9,1368,1
17034,4,4868,1
9845,3,1094,1
...,...,...,...
22,1,27,1148
20,1,25,1148
19,1,24,1148
18,1,23,1148


In [29]:
df.groupby(['loja_id','produto_id']).agg({'data':'count'}).reset_index().sort_values('data')['data'].value_counts().sort_index()

data
1          21
2          26
4           5
5           1
7           8
        ...  
1144       13
1145       13
1146        2
1147        1
1148    29466
Name: count, Length: 1063, dtype: int64

In [39]:
# Descobrir todas as combinações únicas de loja e produto
comb_loja_produto = df[['loja_id', 'produto_id']].drop_duplicates()

In [40]:
# Criar um intervalo de datas cobrindo o período da base
min_data = df['data'].min()
max_data = df['data'].max()
todas_datas = pd.date_range(start=min_data, end=max_data, freq='D')

In [49]:
# Criar o DataFrame com todas as combinações possíveis
df_grade = comb_loja_produto.merge(pd.DataFrame({'data': todas_datas}), how='cross')
df_grade['data'] = df_grade['data'].astype('str')

In [50]:
print(comb_loja_produto.shape[0])
print(comb_loja_produto.shape[0] * 1148)
print(df_grade.shape)

44736
51356928
(51356928, 3)


In [51]:
df_grade.dtypes

loja_id        int64
produto_id     int64
data          object
dtype: object

In [53]:
# Realizar o merge para garantir todas as datas no df original
df_completo = df_grade.merge(df, on=['loja_id', 'produto_id', 'data'], how='left')

In [60]:
df_completo.query('loja_id == 4 and produto_id == 3136').tail(15)

,loja_id,produto_id,data,categoria_id,is_medicamento,curva,estoque,venda,custo,preco
51356913,4,3136,2025-02-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51356914,4,3136,2025-02-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51356915,4,3136,2025-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51356916,4,3136,2025-02-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51356917,4,3136,2025-02-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51356918,4,3136,2025-02-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51356919,4,3136,2025-02-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51356920,4,3136,2025-02-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51356921,4,3136,2025-02-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51356922,4,3136,2025-02-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
comb_loja_produto_data_minima = df_completo.query('venda.notnull()').groupby(['loja_id','produto_id']).agg({'data':'min'}).reset_index()

In [58]:
comb_loja_produto_data_minima.sort_values('data')

,loja_id,produto_id,data
0,1,1,2022-01-01
27194,7,744,2022-01-01
27193,7,743,2022-01-01
27192,7,742,2022-01-01
27191,7,741,2022-01-01
...,...,...,...
12561,3,4830,2025-02-21
10941,3,2602,2025-02-21
11662,3,3615,2025-02-21
11682,3,3642,2025-02-21


In [61]:
df_completo = df_completo.merge(comb_loja_produto_data_minima, on=['loja_id', 'produto_id'], suffixes=('', '_min'))

In [62]:
df_completo = df_completo.query('data >= data_min')

In [63]:
df_completo.shape

(41515461, 11)

In [64]:
df_completo['linhas'] = 1
df_completo['linhas_nulas'] = df_completo.venda.isnull()

In [66]:
nulos_por_loja_produto = df_completo.groupby(['loja_id', 'produto_id']).agg({'linhas':'sum','linhas_nulas':'sum'}).reset_index()

In [68]:
nulos_por_loja_produto['prop'] = nulos_por_loja_produto['linhas_nulas'] / nulos_por_loja_produto['linhas']

In [70]:
nulos_por_loja_produto.query('linhas_nulas > 0').sort_values('prop', ascending = False)

,loja_id,produto_id,linhas,linhas_nulas,prop
15252,4,2768,634,85,0.134069
783,1,908,635,85,0.133858
44209,10,4703,635,85,0.133858
44185,10,4676,635,85,0.133858
401,1,466,635,85,0.133858
...,...,...,...,...,...
44125,10,4607,635,85,0.133858
44147,10,4633,635,85,0.133858
44174,10,4663,635,85,0.133858
44178,10,4668,635,85,0.133858


In [88]:
nulos_por_loja_produto.query('linhas_nulas > 0').sort_values('linhas_nulas', ascending = False)

,loja_id,produto_id,linhas,linhas_nulas,prop
44723,10,5277,635,85,0.133858
91,1,101,635,85,0.133858
172,1,199,635,85,0.133858
197,1,230,635,85,0.133858
278,1,322,635,85,0.133858
...,...,...,...,...,...
593,1,690,635,85,0.133858
505,1,582,635,85,0.133858
440,1,508,635,85,0.133858
431,1,498,635,85,0.133858


In [74]:
nulos_por_loja_produto.query('linhas_nulas > 0').loja_id.value_counts()

loja_id
1     163
6     161
9     149
7     149
10    148
2     145
3     144
5     143
4     142
8     137
Name: count, dtype: int64

In [73]:
nulos_por_loja_produto.query('linhas_nulas > 0').produto_id.value_counts()

produto_id
199     10
230     10
328     10
373     10
349     10
        ..
2070     1
2271     1
4615     1
2657     1
4009     1
Name: count, Length: 199, dtype: int64

In [77]:
df_completo.query('loja_id == 10 and produto_id == 4296')

,loja_id,produto_id,data,categoria_id,is_medicamento,curva,estoque,venda,custo,preco,data_min,linhas,linhas_nulas
49495385,10,4296,2023-05-29,9.0,True,C,8.0,0.0,0.039942,0.097754,2023-05-29,1,False
49495386,10,4296,2023-05-30,9.0,True,C,8.0,0.0,0.039942,0.097754,2023-05-29,1,False
49495387,10,4296,2023-05-31,9.0,True,C,8.0,0.0,0.039942,0.097754,2023-05-29,1,False
49495388,10,4296,2023-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-29,1,True
49495389,10,4296,2023-06-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-29,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49496015,10,4296,2025-02-17,9.0,True,C,15.0,0.0,0.037913,0.054756,2023-05-29,1,False
49496016,10,4296,2025-02-18,9.0,True,C,15.0,0.0,0.037913,0.054756,2023-05-29,1,False
49496017,10,4296,2025-02-19,9.0,True,C,15.0,0.0,0.037913,0.054756,2023-05-29,1,False
49496018,10,4296,2025-02-20,9.0,True,C,15.0,0.0,0.037913,0.054756,2023-05-29,1,False


In [ ]:
df_completo.query('loja_id == 10 and produto_id == 4296').query('venda.isnull()')

,loja_id,produto_id,data,categoria_id,is_medicamento,curva,estoque,venda,custo,preco,data_min,linhas,linhas_nulas
49495388,10,4296,2023-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-29,1,True
49495389,10,4296,2023-06-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-29,1,True
49495390,10,4296,2023-06-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-29,1,True
49495391,10,4296,2023-06-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-29,1,True
49495392,10,4296,2023-06-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-29,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49495468,10,4296,2023-08-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-29,1,True
49495469,10,4296,2023-08-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-29,1,True
49495470,10,4296,2023-08-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-29,1,True
49495471,10,4296,2023-08-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-29,1,True


In [78]:
df_completo.query('loja_id == 10 and produto_id == 4633')

,loja_id,produto_id,data,categoria_id,is_medicamento,curva,estoque,venda,custo,preco,data_min,linhas,linhas_nulas
12968321,10,4633,2023-05-29,3.0,True,B,0.0,0.0,0.021389,0.048865,2023-05-29,1,False
12968322,10,4633,2023-05-30,3.0,True,B,0.0,0.0,0.021389,0.048865,2023-05-29,1,False
12968323,10,4633,2023-05-31,3.0,True,B,0.0,0.0,0.021389,0.048865,2023-05-29,1,False
12968324,10,4633,2023-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-29,1,True
12968325,10,4633,2023-06-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-29,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12968951,10,4633,2025-02-17,3.0,True,B,7.0,1.0,0.041996,0.061673,2023-05-29,1,False
12968952,10,4633,2025-02-18,3.0,True,B,7.0,0.0,0.041996,0.061673,2023-05-29,1,False
12968953,10,4633,2025-02-19,3.0,True,B,7.0,0.0,0.041996,0.061673,2023-05-29,1,False
12968954,10,4633,2025-02-20,3.0,True,B,7.0,0.0,0.041996,0.061673,2023-05-29,1,False


In [79]:
df_completo.query('loja_id == 10 and produto_id == 4633').query('venda.isnull()')

,loja_id,produto_id,data,categoria_id,is_medicamento,curva,estoque,venda,custo,preco,data_min,linhas,linhas_nulas
12968324,10,4633,2023-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-29,1,True
12968325,10,4633,2023-06-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-29,1,True
12968326,10,4633,2023-06-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-29,1,True
12968327,10,4633,2023-06-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-29,1,True
12968328,10,4633,2023-06-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-29,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12968404,10,4633,2023-08-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-29,1,True
12968405,10,4633,2023-08-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-29,1,True
12968406,10,4633,2023-08-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-29,1,True
12968407,10,4633,2023-08-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-29,1,True


In [89]:
df_completo.query('loja_id == 4 and produto_id == 2768')

,loja_id,produto_id,data,categoria_id,is_medicamento,curva,estoque,venda,custo,preco,data_min,linhas,linhas_nulas
9383118,4,2768,2023-05-30,3.0,True,C,2.0,0.0,0.065438,0.110000,2023-05-30,1,False
9383119,4,2768,2023-05-31,3.0,True,C,2.0,0.0,0.065438,0.110000,2023-05-30,1,False
9383120,4,2768,2023-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-30,1,True
9383121,4,2768,2023-06-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-30,1,True
9383122,4,2768,2023-06-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-30,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9383747,4,2768,2025-02-17,3.0,True,C,2.0,0.0,0.081229,0.114669,2023-05-30,1,False
9383748,4,2768,2025-02-18,3.0,True,C,2.0,0.0,0.081229,0.114669,2023-05-30,1,False
9383749,4,2768,2025-02-19,3.0,True,C,2.0,0.0,0.081229,0.114669,2023-05-30,1,False
9383750,4,2768,2025-02-20,3.0,True,C,2.0,0.0,0.081229,0.114669,2023-05-30,1,False


In [90]:
df_completo.query('loja_id == 4 and produto_id == 2768').query('venda.isnull()')

,loja_id,produto_id,data,categoria_id,is_medicamento,curva,estoque,venda,custo,preco,data_min,linhas,linhas_nulas
9383120,4,2768,2023-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-30,1,True
9383121,4,2768,2023-06-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-30,1,True
9383122,4,2768,2023-06-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-30,1,True
9383123,4,2768,2023-06-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-30,1,True
9383124,4,2768,2023-06-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-30,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9383200,4,2768,2023-08-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-30,1,True
9383201,4,2768,2023-08-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-30,1,True
9383202,4,2768,2023-08-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-30,1,True
9383203,4,2768,2023-08-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-30,1,True


In [82]:
datas_nulas_minimas_maximas = df_completo.query('venda.isnull()').groupby(['loja_id', 'produto_id']).agg({'data':['min','max'],'linhas_nulas':'sum'}).reset_index()

In [84]:
datas_nulas_minimas_maximas.columns = ['loja_id','produto_id','data_min','data_max','linhas_nulas']

In [87]:
print(datas_nulas_minimas_maximas.data_min.min())
print(datas_nulas_minimas_maximas.data_min.max())
print(datas_nulas_minimas_maximas.data_max.min())
print(datas_nulas_minimas_maximas.data_max.max())

2023-06-01
2023-06-01
2023-08-24
2023-08-24
